# VacationPy
----


In [107]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame


In [108]:
filepath = ("../WeatherPy/Output File/city_data.csv")
city_data = pd.read_csv(filepath)
city_data.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rosarito,32.33,-117.03,73.40,28.0,1.0,8.05,MX,1.603930e+09
1,1,san ramon,37.78,-121.98,75.00,26.0,0.0,1.01,US,1.603930e+09
2,2,ushuaia,-54.80,-68.30,39.27,86.0,51.0,4.81,AR,1.603930e+09
3,3,saskylakh,71.92,114.08,2.70,90.0,97.0,9.93,RU,1.603930e+09
4,4,kruisfontein,-34.00,24.73,61.00,93.0,67.0,3.04,ZA,1.603930e+09


### Humidity Heatmap


In [109]:
cd_2 = city_data.dropna(how='any')

In [110]:
gmaps.configure(api_key=g_key)

In [111]:
locations = cd_2[['Latitude', 'Longitude']]
weights = cd_2['Humidity']
heat_fig = gmaps.figure()
heat_fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [120]:
nice_weather = cd_2.drop(cd_2[(cd_2['Max Temp'] <= 74) | (cd_2['Max Temp'] >= 80) | (cd_2['Cloudiness'] >= 50) | (cd_2['Wind Speed'] >= 10)| (cd_2['Humidity'] >= 70)].index)

nice_weather.reset_index(drop=True)

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,san ramon,37.78,-121.98,75.00,26.0,0.0,1.01,US,1.603930e+09
1,104,koupela,12.18,-0.35,76.30,19.0,0.0,4.12,BF,1.603931e+09
2,198,gao,16.64,1.64,79.99,15.0,0.0,6.17,ML,1.603931e+09
3,200,abu zabad,12.35,29.25,79.84,26.0,1.0,8.23,SD,1.603931e+09
4,209,boyuibe,-20.42,-63.28,78.51,48.0,5.0,5.44,BO,1.603931e+09
5,405,presidencia roque saenz pena,-26.79,-60.44,75.00,53.0,4.0,7.16,AR,1.603931e+09
6,414,ferkessedougou,9.60,-5.20,75.61,49.0,28.0,2.80,CI,1.603931e+09
7,431,oakley,38.00,-121.71,75.00,26.0,0.0,2.53,US,1.603931e+09
8,516,teseney,15.11,36.66,78.82,45.0,0.0,8.08,ER,1.603931e+09
9,536,maiduguri,11.85,13.16,77.90,25.0,13.0,7.47,NG,1.603931e+09


### Hotel Map


In [121]:
hotel_df = pd.DataFrame(nice_weather[["City", "Country", "Latitude", "Longitude"]].copy())
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(drop=True)


,City,Country,Latitude,Longitude,Hotel Name
0,san ramon,US,37.78,-121.98,
1,koupela,BF,12.18,-0.35,
2,gao,ML,16.64,1.64,
3,abu zabad,SD,12.35,29.25,
4,boyuibe,BO,-20.42,-63.28,
5,presidencia roque saenz pena,AR,-26.79,-60.44,
6,ferkessedougou,CI,9.60,-5.20,
7,oakley,US,38.00,-121.71,
8,teseney,ER,15.11,36.66,
9,maiduguri,NG,11.85,13.16,


In [141]:
for index, row in nice_weather.iterrows():
    city_name = row['City']
    lat = row["Latitude"]
    lng = row["Longitude"]
    params = {
        "radius" : "5000",
        "type" : "lodging",
        "key" : g_key,
        "location" : f"{lat},{lng}"
      }
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    hotels = requests.get(base_url, params=params)
   # convert to json

    hotels = hotels.json()
    
    #print(json.dumps(hotels, indent=4))
    #Grab the first hotel from the results and store the name
    try:        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][1]["name"]
        print(f"Closest hotel to {city_name} is {hotels['results'][1]['name']}")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")




Closest hotel to san ramon is Best Western Danville Sycamore Inn
Closest hotel to koupela is Hotel Le Remblais
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel to boyuibe is Alojamiento camireño
Closest hotel to presidencia roque saenz pena is Hotel Gualok
Closest hotel to ferkessedougou is Hotel le Pelican
Closest hotel to oakley is Sandy Point 3
Closest hotel to teseney is Ferait Hotel
Closest hotel to maiduguri is Nanne and Boi Suites


In [142]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,san ramon,US,37.78,-121.98,Best Western Danville Sycamore Inn
104,koupela,BF,12.18,-0.35,Hotel Le Remblais
198,gao,ML,16.64,1.64,Sidimaiga@.Com.fr
200,abu zabad,SD,12.35,29.25,ابوزبد اعمال وانجا التجارية
209,boyuibe,BO,-20.42,-63.28,Alojamiento camireño
405,presidencia roque saenz pena,AR,-26.79,-60.44,Hotel Gualok
414,ferkessedougou,CI,9.60,-5.20,Hotel le Pelican
431,oakley,US,38.00,-121.71,Sandy Point 3
516,teseney,ER,15.11,36.66,Ferait Hotel
536,maiduguri,NG,11.85,13.16,Nanne and Boi Suites


In [143]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [144]:
# Add marker layer ontop of heat map
hotel_fig=gmaps.figure()
marker_layer = gmaps.marker_layer(locations2,info_box_content=hotel_info)
hotel_fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
hotel_fig.add_layer(marker_layer)

# Display figure
hotel_fig

Figure(layout=FigureLayout(height='420px'))